# Ensemble models

We start of by building ensemble models for our data. The goal is to be able to predict pChembl value based on the drugs structures and target's sequence information. This value allows a number of roughly comparable measures of half-maximal response concentration/potency/affinity to be compared on a negative logarithmic scale. For example, an IC50 measurement of 1nM would have a pChEMBL value of 9. pChEMBL is defined as: -Log(molar IC50, XC50, EC50, AC50, Ki, Kd or Potency).

In [2]:
# standard libraries imported
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
# specific for this task as hand
import pickle
import dask.dataframe as dd
from sklearn.ensemble import RandomForestRegressor
from rdkit.Chem import AllChem
from rdkit import Chem

# Merge files and create train and test dataset

I have downladed PSSM data on our target protein from the NCBI protein blast website I want to test which method of protein featurization works best in ensemble models.

In [4]:
#protein and compound feature vector file
k_sep_norm = pd.read_csv("../cleaned_data/k-sep_normalized.tsv",sep="\t")  #protein
ecfp4 = pd.read_csv("../cleaned_data/ECFP4.tsv",sep="\t")  #compound

In [6]:
#train dataset
df_tr = pd.read_csv("../cleaned_data/traincomps.tsv",sep="\t")

In [7]:
#train dataset with protein-compound feature vectors
df = df_tr.merge(k_sep_norm,on="target_id").merge(ecfp4,on="compound_id")

In [8]:
df.head()

,compound_id,target_id,pchembl_value,k_separated_bigrams_pssm0,k_separated_bigrams_pssm1,k_separated_bigrams_pssm2,k_separated_bigrams_pssm3,k_separated_bigrams_pssm4,k_separated_bigrams_pssm5,k_separated_bigrams_pssm6,...,ECFP4.1015,ECFP4.1016,ECFP4.1017,ECFP4.1018,ECFP4.1019,ECFP4.1020,ECFP4.1021,ECFP4.1022,ECFP4.1023,ECFP4.1024
0,CHEMBL1000,Q02763,4.05,0.132645,0.246746,0.191735,0.210505,0.210938,0.130516,0.154157,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL103667,Q02763,7.70,0.132645,0.246746,0.191735,0.210505,0.210938,0.130516,0.154157,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL103667,P12931,4.46,0.062425,0.136338,0.085795,0.097171,0.050512,0.078817,0.086594,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL103667,P35968,5.50,0.108654,0.241820,0.227927,0.240146,0.144558,0.159763,0.186377,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL103667,P09619,5.96,0.098823,0.214194,0.193479,0.199802,0.115261,0.139842,0.162413,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df.shape

(87769, 1427)

# first attempt using small subset


In [10]:
df_set = df.sample(20000); df_set.shape

(20000, 1427)

In [11]:
y = df_set['pchembl_value']
X = df_set.drop(['compound_id', 'target_id', 'pchembl_value'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from xgboost import XGBRegressor

# # A parameter grid for XGBoost
# params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
# 'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# # Initialize XGB and GridSearch
# xgb = XGBRegressor(nthread=-1, objective="reg:linear") 

# grid = GridSearchCV(xgb, params)
# grid.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])


In [27]:
# Instantiate XGBRegressor
clf = XGBRegressor(objective="reg:linear")

# Fit XGBRegressor
clf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

# Predict on training and test sets
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

[0]	validation_0-rmse:5.99066
[1]	validation_0-rmse:5.41702
[2]	validation_0-rmse:4.90277
[3]	validation_0-rmse:4.44286
[4]	validation_0-rmse:4.03117
[5]	validation_0-rmse:3.66384
[6]	validation_0-rmse:3.33496
[7]	validation_0-rmse:3.04323
[8]	validation_0-rmse:2.78338
[9]	validation_0-rmse:2.55393
[10]	validation_0-rmse:2.35139
[11]	validation_0-rmse:2.17304
[12]	validation_0-rmse:2.01683
[13]	validation_0-rmse:1.88042
[14]	validation_0-rmse:1.76129
[15]	validation_0-rmse:1.65815
[16]	validation_0-rmse:1.5692
[17]	validation_0-rmse:1.49246
[18]	validation_0-rmse:1.42775
[19]	validation_0-rmse:1.37289
[20]	validation_0-rmse:1.32609
[21]	validation_0-rmse:1.28621
[22]	validation_0-rmse:1.25288
[23]	validation_0-rmse:1.2245
[24]	validation_0-rmse:1.20106
[25]	validation_0-rmse:1.1811
[26]	validation_0-rmse:1.16329
[27]	validation_0-rmse:1.14923
[28]	validation_0-rmse:1.13802
[29]	validation_0-rmse:1.12832
[30]	validation_0-rmse:1.11974
[31]	validation_0-rmse:1.11172
[32]	validation_0-rms

In [28]:
import sklearn
print('\nMean Square error" ', sklearn.metrics.mean_squared_error(y_test,test_preds))


Mean Square error"  1.0366595436384993


# Split dataset and fit the model on entire data

In [31]:
y = df['pchembl_value']
X = df.drop(['compound_id', 'target_id', 'pchembl_value'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
# Instantiate XGBRegressor
clf = XGBRegressor(objective="reg:linear")

# Fit XGBRegressor
clf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

# Predict on training and test sets
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

[0]	validation_0-rmse:5.98382
[1]	validation_0-rmse:5.41042
[2]	validation_0-rmse:4.89689
[3]	validation_0-rmse:4.43687
[4]	validation_0-rmse:4.02542
[5]	validation_0-rmse:3.65829
[6]	validation_0-rmse:3.33069
[7]	validation_0-rmse:3.03893
[8]	validation_0-rmse:2.78034
[9]	validation_0-rmse:2.55123
[10]	validation_0-rmse:2.34867
[11]	validation_0-rmse:2.17017
[12]	validation_0-rmse:2.01374
[13]	validation_0-rmse:1.87737
[14]	validation_0-rmse:1.75828
[15]	validation_0-rmse:1.65516
[16]	validation_0-rmse:1.56622
[17]	validation_0-rmse:1.49015
[18]	validation_0-rmse:1.42486
[19]	validation_0-rmse:1.36894
[20]	validation_0-rmse:1.32152
[21]	validation_0-rmse:1.28186
[22]	validation_0-rmse:1.24827
[23]	validation_0-rmse:1.22004
[24]	validation_0-rmse:1.19679
[25]	validation_0-rmse:1.17677
[26]	validation_0-rmse:1.16003
[27]	validation_0-rmse:1.14562
[28]	validation_0-rmse:1.1338
[29]	validation_0-rmse:1.12394
[30]	validation_0-rmse:1.11502
[31]	validation_0-rmse:1.10749
[32]	validation_0-r

In [33]:
print('\nMean Square error" ', sklearn.metrics.mean_squared_error(y_test,test_preds))


Mean Square error"  1.0162572429739574


# run the model using different protein featurization method

In [36]:
data = pd.read_csv('cleaned_data/first_data.csv')
data.head()

,Unnamed: 0,compound_id,target_id,pchembl_value,encode_0,encode_1,encode_2,encode_3,encode_4,encode_5,...,ECFP4.1015,ECFP4.1016,ECFP4.1017,ECFP4.1018,ECFP4.1019,ECFP4.1020,ECFP4.1021,ECFP4.1022,ECFP4.1023,ECFP4.1024
0,0,CHEMBL1000,Q02763,4.05,11,14,12,3,1,12,...,0,0,0,0,0,0,0,0,0,0
1,1,CHEMBL103667,Q02763,7.70,11,14,12,3,1,12,...,0,0,0,0,0,0,0,0,0,0
2,2,CHEMBL103667,P12931,4.46,11,19,12,8,18,12,...,0,0,0,0,0,0,0,0,0,0
3,3,CHEMBL103667,P35968,5.50,11,10,12,18,4,3,...,0,0,0,0,0,0,0,0,0,0
4,4,CHEMBL103667,P09619,5.96,11,16,3,20,19,1,...,0,0,0,0,0,0,0,0,0,0


In [37]:
data.columns

Index(['Unnamed: 0', 'compound_id', 'target_id', 'pchembl_value', 'encode_0',
       'encode_1', 'encode_2', 'encode_3', 'encode_4', 'encode_5',
       ...
       'ECFP4.1015', 'ECFP4.1016', 'ECFP4.1017', 'ECFP4.1018', 'ECFP4.1019',
       'ECFP4.1020', 'ECFP4.1021', 'ECFP4.1022', 'ECFP4.1023', 'ECFP4.1024'],
      dtype='object', length=5028)

In [38]:
y = data['pchembl_value']
X = data.drop(['Unnamed: 0', 'compound_id', 'target_id', 'pchembl_value'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
# Instantiate XGBRegressor
clf = XGBRegressor(objective="reg:linear")

# Fit XGBRegressor
clf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

# Predict on training and test sets
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

[0]	validation_0-rmse:5.98352
[1]	validation_0-rmse:5.41005
[2]	validation_0-rmse:4.89609
[3]	validation_0-rmse:4.43631
[4]	validation_0-rmse:4.02511
[5]	validation_0-rmse:3.65774
[6]	validation_0-rmse:3.32993
[7]	validation_0-rmse:3.0379
[8]	validation_0-rmse:2.77903
[9]	validation_0-rmse:2.54936
[10]	validation_0-rmse:2.34604
[11]	validation_0-rmse:2.16799
[12]	validation_0-rmse:2.01125
[13]	validation_0-rmse:1.87443
[14]	validation_0-rmse:1.75494
[15]	validation_0-rmse:1.65175
[16]	validation_0-rmse:1.56242
[17]	validation_0-rmse:1.48517
[18]	validation_0-rmse:1.41941
[19]	validation_0-rmse:1.36352
[20]	validation_0-rmse:1.31614
[21]	validation_0-rmse:1.27627
[22]	validation_0-rmse:1.2419
[23]	validation_0-rmse:1.21262
[24]	validation_0-rmse:1.18883
[25]	validation_0-rmse:1.16864
[26]	validation_0-rmse:1.15172
[27]	validation_0-rmse:1.13722
[28]	validation_0-rmse:1.1253
[29]	validation_0-rmse:1.11505
[30]	validation_0-rmse:1.10639
[31]	validation_0-rmse:1.09872
[32]	validation_0-rms

In [40]:
print('\nMean Square error" ', sklearn.metrics.mean_squared_error(y_test,test_preds))


Mean Square error"  0.9928025492925805


In [41]:
import joblib

In [43]:
joblib.dump(clf, 'model/xgb_onehot.pkl')

['model/xgb_onehot.pkl']

# random forest test

In [46]:
y = df['pchembl_value']
X = df.drop(['compound_id', 'target_id', 'pchembl_value'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [47]:
#RF regression based model generation and prediction
RFreg = RandomForestRegressor(n_estimators=100,max_features=0.33,random_state=42)

RFreg.fit(X_train, y_train)
pred_result = RFreg.predict(X_test)

In [48]:
print('\nMean Square error" ', sklearn.metrics.mean_squared_error(y_test,pred_result))


Mean Square error"  0.4192608985677407


In [49]:
joblib.dump(RFreg, 'model/rfreg_onehot.pkl')

['model/rfreg_onehot.pkl']